<a href="https://colab.research.google.com/github/rushikeshnakhate/PortfoliOpt/blob/master/multi_dates_post_processing_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from pathlib import Path
import os
output_dir = Path("/content/drive/MyDrive/")
data_dir = Path("/content/drive/MyDrive/data")
import matplotlib.pyplot as plt
# !pip install tqdm
# !pip install PyPortfolioOpt
# !pip install tabulate
# !pip install dtale
# !pip install -q data-table
import scipy.optimize as sci_opt
from tqdm import tqdm  # Import the tqdm progress b
from google.colab import data_table

# !rm -rf output_dir/MyDrive/data
# !rm -rf output_dir/MyDrive/data*

In [16]:
import tarfile

def unzip_files():
  from google.colab import files
  import zipfile

  # After uploading, you can unzip the file
  import zipfile
  import os


  zip_filename = '/content/drive/MyDrive/data.zip'
  # Replace 'data.zip' with the name of your uploaded zip file
  with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
      zip_ref.extractall(output_dir)

  tar_filename = '/content/drive/MyDrive/data.tar.gz'
  # with tarfile.open(tar_filename, 'r:gz') as tar_ref:
  #   tar_ref.extractall(output_dir)

  print("Directory uploaded and extracted successfully to Google Drive.")
unzip_files()

Directory uploaded and extracted successfully to Google Drive.


In [17]:
from datetime import datetime, timedelta
def generate_month_date_ranges(year, months=None):
    """Generate start_date and end_date (without time) for each month in the specified year.

    Args:
        year (int): The year for which to generate the month ranges.
        months (list of int, optional): Specific months to generate ranges for (1-12).

    Returns:
        list of tuples: Each tuple contains the start and end date for the specified months.
    """
    if months is None:
        months = range(1, 13)  # Default to all months if none specified

    month_ranges = []
    for month in months:
        start_date = datetime(year, month, 1).date()
        if month == 12:
            end_date = (datetime(year + 1, 1, 1) - timedelta(days=1)).date()
        else:
            end_date = (datetime(year, month + 1, 1) - timedelta(days=1)).date()
        month_ranges.append((start_date, end_date))

    return month_ranges

def create_current_month_directory(start_date, output_dir):
    current_month = start_date.strftime("%Y%m")
    current_month_dir = output_dir / current_month
    # current_month_dir.rmdir(parents=True, exist_ok=True)
    return current_month_dir

In [18]:
from google.colab import data_table

# Assuming df is your DataFrame
def clean_metadata(value):
    if isinstance(value, pd.Series):
        return value.values  # Extract just the values
    elif isinstance(value, list):
        return [str(v) for v in value]  # Convert each element to string for clarity
    return value

def extract_value(value):
    # Check if the value is a list and has only one element, then extract it
    # print(type(value))
    if isinstance(value, (np.ndarray, list)) and len(value) == 1:
        return value[0]
    # elif isinstance(value, tuple) and len(value) >= 2:
    #   return value[1]
    return value  # Return the value as-is if not a list



In [19]:
def load_data_and_plot(start_date, data_dir, plot_func):
    """
    Function to load data for a specific date range and plot using the given plot function.

    Args:
    start_date (datetime): The start date of the month for data loading.
    data_dir (str): The directory where the data is stored.
    plot_func (function): A function to plot the data (either bar plot or 3D plot).
    """
    current_month_dir = create_current_month_directory(start_date, data_dir)
    data_pkl_filepath = os.path.join(current_month_dir, "data.pkl")
    expected_return_pkl_filepath = os.path.join(current_month_dir, "expected_return.pkl")
    optimization_pkl_filepath = os.path.join(current_month_dir, "optimization.pkl")
    post_processing_weight_pkl_filepath = os.path.join(current_month_dir, "post_processing_weight.pkl")


    # Load data
    data = pd.read_pickle(data_pkl_filepath)
    expected_return_df = pd.read_pickle(expected_return_pkl_filepath)
    optimization_df = pd.read_pickle(optimization_pkl_filepath)
    post_processing_df = pd.read_pickle(post_processing_weight_pkl_filepath)


    # Apply this function to every cell in the DataFrame
    df1 = post_processing_df.applymap(clean_metadata)
    optimized_df_cleaned = df1.applymap(extract_value)

    display(post_processing_df)
    # Format the month as 'YYYY-MM' for the title
    current_month = start_date.strftime('%Y-%m')
    # Call the provided plot function
    # print("using optimization_pkl_filepath=optimization_pkl_filepath{}".format(optimization_pkl_filepath))
    return plot_func(optimized_df_cleaned)

In [20]:
# Update the ranking functions to return the top 10 for best and worst portfolios
target_columns = ['Sharpe Ratio','Annual Volatility','Expected Return Type', 'Risk Model', 'Optimizer', 'Expected Annual Return', 'Weights','Allocation_GreedyPortfolio_weight','Allocation_LpPortfolio_weight','Allocation_GreedyPortfolio_remaining_amount','Allocation_LpPortfolio_remaining_amount']
# target_columns = ['Sharpe Ratio','Annual Volatility','Expected Return Type', 'Risk Model', 'Optimizer', 'Expected Annual Return',  'Weights','Allocation_GreedyPortfolio_weight','Allocation_LpPortfolio_weight']
# Function to rank portfolios based on 'Expected Annual Return', 'GreedyPortfolio_Return', and 'LpPortfolio_Return'
def rank_by_annual_return(df1):
    df = df1.copy()
    # Sort by 'Expected Annual Return'
    df_ranked = df[target_columns]
    best_portfolio_return = df_ranked.nlargest(10, 'Expected Annual Return')  # Top 10 highest return
    worst_portfolio_return = df_ranked.nsmallest(10, 'Expected Annual Return')  # Top 10 lowest return

    return best_portfolio_return, worst_portfolio_return

# Function to rank portfolios based on 'Annual Volatility', 'GreedyPortfolio_Volatility', and 'LpPortfolio_Volatility'
def rank_by_volatility(df1):
    df = df1.copy()
    # Sort by 'Annual Volatility'
    df_ranked = df[target_columns]
    best_portfolio_vol = df_ranked.nsmallest(10, 'Annual Volatility')  # Top 10 smallest volatility (best)
    worst_portfolio_vol = df_ranked.nlargest(10, 'Annual Volatility')  # Top 10 largest volatility (worst)

    return best_portfolio_vol, worst_portfolio_vol

# Function to rank portfolios based on 'Sharpe Ratio', 'GreedyPortfolio_Sharpe', and 'LpPortfolio_Sharpe'
def rank_by_sharpe_ratio(df1):
    df = df1.copy()
    # Sort by 'Sharpe Ratio'
    df_ranked = df[target_columns]
    best_portfolio_sharpe = df_ranked.nlargest(10, 'Sharpe Ratio')  # Top 10 highest Sharpe ratios (best)
    worst_portfolio_sharpe = df_ranked.nsmallest(10, 'Sharpe Ratio')  # Top 10 lowest Sharpe ratios (worst)

    return best_portfolio_sharpe, worst_portfolio_sharpe

# # Assuming filtered_df is already defined, executing the updated functions
# best_return, worst_return = rank_by_annual_return(filtered_df)
# best_volatility, worst_volatility = rank_by_volatility(filtered_df)
# best_sharpe, worst_sharpe = rank_by_sharpe_ratio(filtered_df)

# best_return, worst_return, best_volatility, worst_volatility, best_sharpe, worst_sharpe


# # # Executing the functions
# best_return, worst_return = rank_by_annual_return(filtered_df)
# best_volatility, worst_volatility = rank_by_volatility(filtered_df)
# best_sharpe, worst_sharpe = rank_by_sharpe_ratio(filtered_df)

In [22]:
if __name__ == "__main__":
    year = 2023
    month_ranges = generate_month_date_ranges(year)
    for start_date, end_date in month_ranges:
      print("start_date={},end_date{}".format(start_date, end_date))
      top_best, bottom_worst  = load_data_and_plot(start_date, data_dir, rank_by_annual_return)
      data_table.DataTable(top_best)

start_date=2023-01-01,end_date2023-01-31


<ipython-input-19-9da2d3afde02>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = post_processing_df.applymap(clean_metadata)
<ipython-input-19-9da2d3afde02>:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  optimized_df_cleaned = df1.applymap(extract_value)


,Expected Return Type,Risk Model,Optimizer,Weights,Expected Annual Return,Annual Volatility,Sharpe Ratio,Allocation_GreedyPortfolio_weight,Allocation_GreedyPortfolio_remaining_amount,Allocation_LpPortfolio_weight,Allocation_LpPortfolio_remaining_amount
0,CAGRMeanHistorical,SampleCovariance,pyPortfolioOptFrontier,"{'ASIANPAINT.NS': 0.0, 'BAJAJFINSV.NS': 0.0, '...",2.423074,0.175651,13.680925,"{'HCLTECH.NS': 857, 'HDFCLIFE.NS': 149}",521.274353,"{'HCLTECH.NS': 857, 'HDFCLIFE.NS': 149}",521.273308
1,CAGRMeanHistorical,SampleCovariance,pyPortfolioOptFrontierWithShortPosition,"{'ASIANPAINT.NS': -0.59827, 'BAJAJFINSV.NS': -...",4.729312,0.085072,55.356886,"{'HCLTECH.NS': 162, 'NTPC.NS': 1092, 'TITAN.NS...",372.390259,"{'BAJFINANCE.NS': 14, 'BPCL.NS': 418, 'DIVISLA...",372.381857
2,CAGRMeanHistorical,SampleCovariance,MVRiskFolioOptimizer,"{'ASIANPAINT.NS': 2.4882307362325963e-12, 'BAJ...",1.156757,0.164461,7.033624,"{'HCLTECH.NS': 782, 'HDFCLIFE.NS': 286, 'TATAS...",96.27774,"{'HCLTECH.NS': 782, 'HDFCLIFE.NS': 286}",362.316547
3,CAGRMeanHistorical,SampleCovariance,MADRiskFolioOptimizer,"{'ASIANPAINT.NS': 6.098921410656362e-11, 'BAJA...",1.070182,0.158650,6.745561,"{'HCLTECH.NS': 758, 'DIVISLAB.NS': 46, 'HDFCLI...",30.649109,"{'DIVISLAB.NS': 46, 'HCLTECH.NS': 758, 'HDFCLI...",30.645584
4,CAGRMeanHistorical,SampleCovariance,MSVRiskFolioOptimizer,"{'ASIANPAINT.NS': 3.802148444610364e-09, 'BAJA...",1.289586,0.187518,6.877133,"{'HCLTECH.NS': 924, 'HDFCLIFE.NS': 27, 'BPCL.N...",13.480865,"{'HCLTECH.NS': 924, 'HDFCLIFE.NS': 27}",437.438663
...,...,...,...,...,...,...,...,...,...,...,...
1870,BlackLitterman,CopulaRiskModel,MDDRiskFolioOptimizer,"{'ASIANPAINT.NS': 4.1595296851517936e-11, 'BAJ...",0.660370,0.118211,5.586359,"{'HCLTECH.NS': 439, 'TATASTEEL.NS': 2210, 'DIV...",387.336868,"{'BPCL.NS': 1, 'DIVISLAB.NS': 43, 'HCLTECH.NS'...",575.159440
1871,BlackLitterman,CopulaRiskModel,ADDRiskFolioOptimizer,"{'ASIANPAINT.NS': 1.2948333164800339e-09, 'BAJ...",1.303207,0.190290,6.848514,"{'HCLTECH.NS': 939, 'TATASTEEL.NS': 1}",122.901779,{'HCLTECH.NS': 939},235.617786
1872,BlackLitterman,CopulaRiskModel,CDaRRiskFolioOptimizer,"{'ASIANPAINT.NS': 5.449702703227784e-10, 'BAJA...",0.660370,0.118211,5.586359,"{'HCLTECH.NS': 439, 'TATASTEEL.NS': 2210, 'DIV...",520.991776,"{'BPCL.NS': 1, 'DIVISLAB.NS': 43, 'HCLTECH.NS'...",575.145013
1873,BlackLitterman,CopulaRiskModel,UCIRiskFolioOptimizer,"{'ASIANPAINT.NS': 4.656686215575346e-10, 'BAJA...",1.285589,0.187984,6.838835,"{'HCLTECH.NS': 926, 'TATASTEEL.NS': 117, 'HDFC...",12.379013,"{'HCLTECH.NS': 927, 'TATASTEEL.NS': 115}",48.719291


start_date=2023-02-01,end_date2023-02-28


<ipython-input-19-9da2d3afde02>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = post_processing_df.applymap(clean_metadata)
<ipython-input-19-9da2d3afde02>:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  optimized_df_cleaned = df1.applymap(extract_value)


...

<ipython-input-19-9da2d3afde02>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = post_processing_df.applymap(clean_metadata)
<ipython-input-19-9da2d3afde02>:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  optimized_df_cleaned = df1.applymap(extract_value)


,Expected Return Type,Risk Model,Optimizer,Weights,Expected Annual Return,Annual Volatility,Sharpe Ratio,Allocation_GreedyPortfolio_weight,Allocation_GreedyPortfolio_remaining_amount,Allocation_LpPortfolio_weight,Allocation_LpPortfolio_remaining_amount
0,CAGRMeanHistorical,SampleCovariance,pyPortfolioOptFrontier,"{'ASIANPAINT.NS': 0.0, 'BAJAJFINSV.NS': 0.0, '...",5.532073,0.061464,89.679346,"{'BPCL.NS': 1390, 'HDFCLIFE.NS': 372, 'NTPC.NS...",496.602051,"{'BPCL.NS': 1391, 'DIVISLAB.NS': 42, 'HDFCLIFE...",47.843747
1,CAGRMeanHistorical,SampleCovariance,pyPortfolioOptFrontierWithShortPosition,"{'ASIANPAINT.NS': 0.10667, 'BAJAJFINSV.NS': -0...",4.752574,NaN,NaN,"{'HCLTECH.NS': 189, 'NTPC.NS': 549, 'HDFCLIFE....",516.092880,"{'ASIANPAINT.NS': 12, 'BAJFINANCE.NS': 3, 'BPC...",459.111371
2,CAGRMeanHistorical,SampleCovariance,MVRiskFolioOptimizer,"{'ASIANPAINT.NS': 2.1085314982052378e-10, 'BAJ...",1.630457,0.055732,29.255338,"{'NTPC.NS': 952, 'HDFCLIFE.NS': 330, 'BPCL.NS'...",86.695007,"{'BPCL.NS': 1007, 'DIVISLAB.NS': 39, 'HCLTECH....",381.849378
3,CAGRMeanHistorical,SampleCovariance,MADRiskFolioOptimizer,"{'ASIANPAINT.NS': 1.4959767973567167e-09, 'BAJ...",1.615487,0.057989,27.858263,"{'BPCL.NS': 1085, 'JSWSTEEL.NS': 234, 'NTPC.NS...",186.086517,"{'BHARTIARTL.NS': 22, 'BPCL.NS': 1086, 'DIVISL...",990.542722
4,CAGRMeanHistorical,SampleCovariance,MSVRiskFolioOptimizer,"{'ASIANPAINT.NS': 1.852395024948641e-08, 'BAJA...",1.657192,0.058529,28.314151,"{'HDFCLIFE.NS': 365, 'BPCL.NS': 1145, 'DIVISLA...",82.275024,"{'BPCL.NS': 1144, 'DIVISLAB.NS': 52, 'HCLTECH....",483.495018
...,...,...,...,...,...,...,...,...,...,...,...
2125,BlackLitterman,CopulaRiskModel,MDDRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.03898455739030954, 'BAJAJF...",1.141826,0.063947,17.855785,"{'TATASTEEL.NS': 958, 'DIVISLAB.NS': 28, 'HDFC...",12.634857,"{'ASIANPAINT.NS': 12, 'BAJAJFINSV.NS': 28, 'BA...",42.506793
2126,BlackLitterman,CopulaRiskModel,ADDRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.02677887466476765, 'BAJAJF...",1.429909,0.063027,22.687351,"{'BPCL.NS': 887, 'DIVISLAB.NS': 42, 'HDFCLIFE....",76.410889,"{'ASIANPAINT.NS': 9, 'BAJAJFINSV.NS': 13, 'BAJ...",45.547418
2127,BlackLitterman,CopulaRiskModel,CDaRRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.04218516900727162, 'BAJAJF...",1.152475,0.061890,18.621369,"{'HDFCLIFE.NS': 166, 'KOTAKBANK.NS': 62, 'TATA...",112.552856,"{'ASIANPAINT.NS': 14, 'BAJAJFINSV.NS': 21, 'BA...",111.144038
2128,BlackLitterman,CopulaRiskModel,UCIRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.03148258103060203, 'BAJAJF...",1.164814,0.066292,17.570935,"{'TATASTEEL.NS': 1022, 'DIVISLAB.NS': 34, 'HDF...",178.302048,"{'ASIANPAINT.NS': 10, 'BAJAJFINSV.NS': 26, 'BA...",113.522990


start_date=2023-12-01,end_date2023-12-31


<ipython-input-19-9da2d3afde02>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = post_processing_df.applymap(clean_metadata)
<ipython-input-19-9da2d3afde02>:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  optimized_df_cleaned = df1.applymap(extract_value)


,Expected Return Type,Risk Model,Optimizer,Weights,Expected Annual Return,Annual Volatility,Sharpe Ratio,Allocation_GreedyPortfolio_weight,Allocation_GreedyPortfolio_remaining_amount,Allocation_LpPortfolio_weight,Allocation_LpPortfolio_remaining_amount
0,CAGRMeanHistorical,SampleCovariance,pyPortfolioOptFrontier,"{'ASIANPAINT.NS': 0.12232, 'BAJAJFINSV.NS': 0....",2.903896,1.261298e-01,2.286450e+01,"{'HDFCBANK.NS': 276, 'NTPC.NS': 779, 'ASIANPAI...",133.404022,"{'ASIANPAINT.NS': 36, 'HCLTECH.NS': 53, 'HDFCB...",299.049126
1,CAGRMeanHistorical,SampleCovariance,pyPortfolioOptFrontierWithShortPosition,"{'ASIANPAINT.NS': 0.81545, 'BAJAJFINSV.NS': -0...",2.728612,1.436068e-07,1.886131e+07,"{'ASIANPAINT.NS': 61, 'BHARTIARTL.NS': 143, 'H...",101.470169,"{'ASIANPAINT.NS': 61, 'BAJFINANCE.NS': 3, 'BHA...",349.185960
2,CAGRMeanHistorical,SampleCovariance,MVRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.35500859703078375, 'BAJAJF...",1.008328,8.852052e-02,1.139090e+01,"{'ASIANPAINT.NS': 105, 'HDFCBANK.NS': 201, 'CI...",174.047607,"{'ASIANPAINT.NS': 105, 'CIPLA.NS': 106, 'HDFCB...",1064.007120
3,CAGRMeanHistorical,SampleCovariance,MADRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.3130353443519393, 'BAJAJFI...",0.937463,8.530663e-02,1.098934e+01,"{'HDFCBANK.NS': 210, 'ASIANPAINT.NS': 93, 'CIP...",178.031982,"{'ASIANPAINT.NS': 93, 'BAJFINANCE.NS': 5, 'CIP...",178.016568
4,CAGRMeanHistorical,SampleCovariance,MSVRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.31865943211614917, 'BAJAJF...",0.940737,8.798173e-02,1.069242e+01,"{'HDFCBANK.NS': 254, 'ASIANPAINT.NS': 95, 'CIP...",292.417068,"{'ASIANPAINT.NS': 95, 'CIPLA.NS': 96, 'HDFCBAN...",42.531989
...,...,...,...,...,...,...,...,...,...,...,...
2098,BlackLitterman,RegimeSwitchingRiskModel,MDDRiskFolioOptimizer,"{'ASIANPAINT.NS': 4.943435534848225e-10, 'BAJA...",0.992441,1.071797e-01,9.259599e+00,"{'HDFCBANK.NS': 273, 'JSWSTEEL.NS': 215, 'CIPL...",205.461838,"{'CIPLA.NS': 127, 'HDFCBANK.NS': 274, 'HDFCLIF...",92.433139
2099,BlackLitterman,RegimeSwitchingRiskModel,ADDRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.26849385996200653, 'BAJAJF...",0.925040,9.080646e-02,1.018694e+01,"{'HDFCBANK.NS': 272, 'ASIANPAINT.NS': 80, 'HDF...",136.343033,"{'ASIANPAINT.NS': 80, 'CIPLA.NS': 36, 'HDFCBAN...",54.779329
2100,BlackLitterman,RegimeSwitchingRiskModel,CDaRRiskFolioOptimizer,"{'ASIANPAINT.NS': 2.2807454070845896e-09, 'BAJ...",0.992441,1.071797e-01,9.259599e+00,"{'HDFCBANK.NS': 273, 'JSWSTEEL.NS': 215, 'CIPL...",205.461838,"{'CIPLA.NS': 127, 'HDFCBANK.NS': 274, 'HDFCLIF...",92.423966
2101,BlackLitterman,RegimeSwitchingRiskModel,UCIRiskFolioOptimizer,"{'ASIANPAINT.NS': 0.18424385452611527, 'BAJAJF...",0.882618,9.389638e-02,9.399919e+00,"{'HDFCBANK.NS': 280, 'ASIANPAINT.NS': 55, 'ONG...",28.023056,"{'ASIANPAINT.NS': 55, 'CIPLA.NS': 61, 'HDFCBAN...",58.821294


In [23]:
# data_pkl_filepath = os.path.join(data_dir, "202301/data.pkl")
# data_df =  pd.read_pickle(data_pkl_filepath)
# data_table.DataTable(data_df)

In [10]:
# data_table.DataTable(performance_df_cleaned.head(10))

In [11]:
# data_table.DataTable(filtered_df)

In [12]:
# print(filtered_df.iloc[1137])